In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#载入训练和测试的数据

train_data=pd.read_csv("train.csv",index_col=0)
test_data=pd.read_csv("test.csv",index_col=0)


In [3]:
#查看数据，注意训练数据多一列，就是最后的价格标签用作训练目标

print(train_data.shape)
train_data.head()



(1460, 80)


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
test_data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [5]:
#准备训练的LABEL
train_y=np.log1p(train_data.pop("SalePrice"))
#label需要转化成int,不然分类训练有可能报错

train_y=train_y.values
train_y=train_y.astype("int")




In [6]:
#把训练和测试的输入数据联在一起方便做数据预处理

all_x=pd.concat((train_data,test_data),axis=0)




In [7]:
#对于内容项目进行one-hot-coding
all_dummy_data=pd.get_dummies(all_x)

all_dummy_data.head()
#all_dummy_data.isnull().sum().sort_values(ascending=False).head(10)


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
1,60,65.0,8450,7,5,2003,2003,196.0,706.0,0.0,...,0,0,0,1,0,0,0,0,1,0
2,20,80.0,9600,6,8,1976,1976,0.0,978.0,0.0,...,0,0,0,1,0,0,0,0,1,0
3,60,68.0,11250,7,5,2001,2002,162.0,486.0,0.0,...,0,0,0,1,0,0,0,0,1,0
4,70,60.0,9550,7,5,1915,1970,0.0,216.0,0.0,...,0,0,0,1,1,0,0,0,0,0
5,60,84.0,14260,8,5,2000,2000,350.0,655.0,0.0,...,0,0,0,1,0,0,0,0,1,0


In [8]:
#计算平均值来用于替代NAN

mean_cols=all_dummy_data.mean()




In [9]:
#用平均值来填充空缺
all_dummy_data=all_dummy_data.fillna(mean_cols)


In [10]:
#对于数值列进行归一化处理
numeric_cols=all_x.columns[all_x.dtypes!="object"]
numeric_cols


Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold'],
      dtype='object')

In [11]:
#计算平均值和方差来对数据进行归一化处理
numeric_col_means=all_dummy_data.loc[:,numeric_cols].mean()
numeric_col_std=all_dummy_data.loc[:,numeric_cols].std()

all_dummy_data.loc[:,numeric_cols]=(all_dummy_data.loc[:,numeric_cols]-numeric_col_means)/numeric_col_std

all_dummy_data.head()




,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
1,0.067320,-0.202033,-0.217841,0.646073,-0.507197,1.046078,0.896679,0.525112,0.580807,-0.29308,...,0,0,0,1,0,0,0,0,1,0
2,-0.873466,0.501785,-0.072032,-0.063174,2.187904,0.154737,-0.395536,-0.572152,1.177910,-0.29308,...,0,0,0,1,0,0,0,0,1,0
3,0.067320,-0.061269,0.137173,0.646073,-0.507197,0.980053,0.848819,0.334770,0.097856,-0.29308,...,0,0,0,1,0,0,0,0,1,0
4,0.302516,-0.436639,-0.078371,0.646073,-0.507197,-1.859033,-0.682695,-0.572152,-0.494856,-0.29308,...,0,0,0,1,1,0,0,0,0,0
5,0.067320,0.689469,0.518814,1.355319,-0.507197,0.947040,0.753100,1.387248,0.468851,-0.29308,...,0,0,0,1,0,0,0,0,1,0


In [12]:
#获得numpy数组的格式用于模型训练

train_x=all_dummy_data.loc[train_data.index]
test_x=all_dummy_data.loc[test_data.index]

train_x=train_x.values
test_x=test_x.values


print(train_x.shape)
print(train_y.shape)
print(test_x.shape)






(1460, 288)
(1460,)
(1459, 288)


In [13]:
#使用基础的knn模型

from sklearn.neighbors import KNeighborsClassifier



In [14]:
#训练模型
knn=KNeighborsClassifier(n_neighbors=10)
knn.fit(train_x,train_y)


KNeighborsClassifier(n_neighbors=10)

In [15]:
#预测模型

predictions=knn.predict(test_x)

predictions=np.expm1(predictions)



In [16]:
#生成并存待上交的数据格式

submission_data = pd.DataFrame(data={"Id":test_data.index,"SalePrice":predictions})

submission_data.to_csv("mysubmission.csv",index=False)



In [17]:
#搭建一个高级的ensemble model

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor





In [18]:
#设定并训练模型

ridge=Ridge(alpha=15)
rf=RandomForestRegressor(n_estimators=500, max_features=0.3)

ridge.fit(train_x,train_y)
rf.fit(train_x,train_y)







RandomForestRegressor(max_features=0.3, n_estimators=500)

In [19]:
#ensemble预测，组合预测

ridge_prediction=np.expm1(ridge.predict(test_x))
rf_prediction=np.expm1(rf.predict(test_x))

ensemble_predictions=(ridge_prediction+rf_prediction)/2





In [20]:
submission_data = pd.DataFrame(data={"Id":test_data.index,"SalePrice":ensemble_predictions})

submission_data.to_csv("mysubmissionRev1.csv",index=False)